In [2]:
import scanpy as sc 
import pandas as pd 
import helper_functions

In [3]:
adata= sc.read_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_unfiltered.h5ad')

In [10]:
adata_celltypist= pd.read_csv('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/all_samples_predictions_unfiltered.csv')

In [11]:
adata_celltypist

,Cell_ID,Sanger_ID,predicted_labels,over_clustering,majority_voting
0,GGTCGGTTCAATTACG-HCA_BN_F12482363,HCA_BN_F12482363,Dermal fibroblasts,0,Dermal fibroblasts
1,AAAGCCGCATCAGTAT-HCA_BN_F12482363,HCA_BN_F12482363,MYH3+ myocytes,1,MYH3+ myocytes
2,CGACAAGCACATTGCA-HCA_BN_F12482363,HCA_BN_F12482363,MYH3+ myocytes,2,Endothelial cells
3,TTCCCACAGTTGGCCA-HCA_BN_F12482363,HCA_BN_F12482363,MYH3+ myocytes,2,Endothelial cells
4,TATCGCACACCTAATG-HCA_BN_F12482363,HCA_BN_F12482363,MYH3+ myocytes,0,Dermal fibroblasts
...,...,...,...,...,...
3012052,TGCTGCTGTCACTTCC-5478STDY7850149,5478STDY7850149,Photoreceptor,119,Photoreceptor
3012053,GATGAGGTCACATGCA-5478STDY7850149,5478STDY7850149,Photoreceptor,120,Photoreceptor
3012054,TGCGGGTAGAGGTTAT-5478STDY7850149,5478STDY7850149,Photoreceptor,119,Photoreceptor
3012055,CCTATTATCCGAGCCA-5478STDY7850149,5478STDY7850149,Photoreceptor,119,Photoreceptor


In [5]:
metadata= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/sample_metadata/COMBINED_BONE_CROSS_ORGAN_METADATA.csv',index_col=0)

In [6]:
import pandas as pd

def add_metadata_to_obs(
    adata,
    df: pd.DataFrame,
    df_key: str,
    obs_key: str = None,
):
    """
    Merge EVERY column of `df` into `adata.obs` by matching
      df[df_key]  ↔  adata.obs[obs_key] (or adata.obs_names if obs_key is None).

    Keeps duplicates in df_key by dropping all but the first.
    Leaves adata.obs index untouched.
    """

    # 1. Sanity checks
    assert df_key in df.columns, f"{df_key!r} not in metadata DataFrame"
    if obs_key is not None:
        assert obs_key in adata.obs.columns, f"{obs_key!r} not in adata.obs"

    # 2. Deduplicate metadata on the key, keep only first
    df_unique = df.drop_duplicates(subset=[df_key], keep="first").copy()

    # 3. Build a lookup Series indexed by the key
    df_indexed = df_unique.set_index(df_key)

    # 4. Pick the “left” key from adata.obs
    if obs_key is None:
        # map from obs_names
        left_index = pd.Series(adata.obs_names, index=adata.obs_names, name=df_key)
    else:
        # map from an existing obs column
        left_index = adata.obs[obs_key].rename(df_key)

    # 5. For every column in df_indexed (including df_key itself), map into adata.obs
    for col in df_indexed.columns:
        adata.obs[col] = left_index.map(df_indexed[col])

    # 6. Report if anything expected went missing
    missing = adata.obs[df_indexed.columns].isna().any(axis=1).sum()
    if missing:
        print(f"⚠️  {missing} cells did not get full metadata from `{df_key}` join")

    return adata


In [7]:
 #)add metadata keyed by Sample_ID → obs['Sanger_ID']
add_metadata_to_obs(adata, metadata,   df_key='Sample_ID', obs_key='Sanger_ID')


⚠️  3012057 cells did not get full metadata from `Sample_ID` join


AnnData object with n_obs × n_vars = 3012057 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_pf', 'S

In [13]:
add_metadata_to_obs(adata, adata_celltypist,   df_key='Cell_ID')


AnnData object with n_obs × n_vars = 3012057 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_pf', 'S

In [14]:
adata

AnnData object with n_obs × n_vars = 3012057 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_pf', 'S

In [15]:
samples=list(adata.obs["Sanger_ID"].unique())

In [16]:
# import os
# import pandas as pd

# # 1.  list of sample names
# samples=list(adata.obs["Sanger_ID"].unique())

# # 2. Base directory where each sample has its own subfolder
# base_dir = "/lustre/scratch126/cellgen/haniffa/sm54/BoneAtlasProject/data/"

# # 3. Container for each sample’s DataFrame
# dfs = []

# for sample in samples:
#     # Construct the expected filename
#     fname = f"predicted_labels_Human_WholeEmbryo_Public_{sample}.csv"
#     fpath = os.path.join(base_dir, sample, fname)
    
#     # Read it in
#     df = pd.read_csv(fpath)
    
#     # (Optional) Tag the DataFrame with its sample name
#     df["Sample_ID"] = sample
    
#     dfs.append(df)

# # 4. Merge all into one DataFrame
# all_labels = pd.concat(dfs, ignore_index=True)

# # 5. Inspect

# (all_labels.shape)
# all_labels.head()


In [17]:
#all_labels

In [18]:
#add_metadata_to_obs(adata, all_labels,   df_key='Cell_ID')



In [79]:
# Add Celltypist metadata levels 

In [19]:
annotation_levels= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/CellTypist_Hierarchy_Levels3_2_1.csv', index_col=0)

In [20]:
# assume you already have:
#   annotation_levels: a pd.DataFrame with columns ["Level1","Level2","Level3"]
#   adata: your AnnData with obs cols "predicted_labels" and "majority_voting"

# 1) make the two mappings
lvl1_map = annotation_levels.set_index("Level3")["Level1"].to_dict()
lvl2_map = annotation_levels.set_index("Level3")["Level2"].to_dict()

# 2) apply to both columns
for src in ["predicted_labels", "majority_voting"]:
    adata.obs[f"{src}_Level1"] = adata.obs[src].map(lvl1_map)
    adata.obs[f"{src}_Level2"] = adata.obs[src].map(lvl2_map)


In [28]:
adata.write_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_unfiltered_with_metadata.h5ad')

... storing 'Sanger_ID' as categorical
... storing 'Organ' as categorical
... storing '10X_Chemistry' as categorical
... storing 'Accession_ID' as categorical
... storing 'Age' as categorical
... storing 'Age(PCW)' as categorical
... storing 'Dataset' as categorical
... storing 'Donor_ID' as categorical
... storing 'FACS_Sorting' as categorical
... storing 'Anatomical_Site' as categorical
... storing 'Method' as categorical
... storing 'Multiplexed' as categorical
... storing 'Organism' as categorical
... storing 'Study' as categorical
... storing 'Technology' as categorical
... storing 'Organ.1' as categorical
... storing 'predicted_labels' as categorical
... storing 'majority_voting' as categorical
... storing 'predicted_labels_Level1' as categorical
... storing 'predicted_labels_Level2' as categorical
... storing 'majority_voting_Level1' as categorical
... storing 'majority_voting_Level2' as categorical


In [29]:
(adata.obs['majority_voting_Level1'].value_counts())

majority_voting_Level1
Mesenchymal       1656559
Stroma             456937
Neural             413564
Erythroid          278208
Haematopoietic     206789
Name: count, dtype: int64

In [30]:
(adata.obs['majority_voting_Level1'].value_counts())

majority_voting_Level1
Mesenchymal       1656559
Stroma             456937
Neural             413564
Erythroid          278208
Haematopoietic     206789
Name: count, dtype: int64

In [31]:
adata.obs['predicted_labels_Level2'].value_counts()

predicted_labels_Level2
Mesenchymal       792280
Neural            494410
Chondrocyte       319609
Muscle            316703
Fibroblast        295832
Erythroid         292541
Myeloid           135003
Endothelial       101284
Epithelial         85664
Lymphoid           80568
Stroma             32595
Progenitor         22313
Gonadal            15610
MK                 12764
Haematopoietic     12639
Endocrine           1911
Liver                330
Gut                    1
Name: count, dtype: int64

# Filter out low cell count, low gene count and high mitochondria count cells 

In [1]:
import scanpy as sc 
import pandas as pd 
import helper_functions

ModuleNotFoundError: No module named 'helper_functions'

In [ ]:
adata= sc.read_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_unfiltered_with_metadata.h5ad')

In [3]:
adata[adata.obs['mt_outlier']==True]

View of AnnData object with n_obs × n_vars = 243622 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_

In [5]:
import numpy as np
from scipy.stats import median_abs_deviation

# 1) make sure you're using the correct metric – usually Scanpy gives you 'pct_counts_mt'
metric = "log1p_total_counts_mt"

# 2) compute the per‐sample threshold
def upper_thresh(x, nmads=3):
    return x.median() + nmads * median_abs_deviation(x)

# this will give you, for each row, the threshold of its sample
th = (
    adata
    .obs
    .groupby("Sanger_ID")[metric]
    .transform(lambda x: upper_thresh(x, nmads=3))
)

# 3) flag anything above that threshold
adata.obs["mt_high_outlier"] = adata.obs[metric] > th


In [6]:
adata[adata.obs['mt_high_outlier']==True]

View of AnnData object with n_obs × n_vars = 243622 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_

In [7]:
conditions = [
    (adata.obs['total_counts_outlier']== True),
    (adata.obs['gene_counts_outlier'] == True),
     (adata.obs['mt_outlier'] == True),
    (
    (adata.obs['gene_counts_outlier']== False) & (adata.obs['gene_counts_outlier'] == False) & (adata.obs['mt_high_outlier']==False))
   
]
values = ['Low_total_count', 'Low_nFeature', "High_Mito",'Pass']

In [8]:
adata.obs['QC'] = np.select(conditions, values, default="Pass")

In [2]:
293301 + 86213 + 139239

518753

In [3]:
2493304 + 518753

3012057

In [9]:
adata.obs['QC'].value_counts()

QC
Pass               2493304
Low_total_count     293301
High_Mito           139239
Low_nFeature         86213
Name: count, dtype: int64

In [10]:
adata.obs["mt_high_outlier"].value_counts()

mt_high_outlier
False    2768435
True      243622
Name: count, dtype: int64

In [11]:
pd.crosstab(adata.obs['mt_high_outlier'],adata.obs['QC'])

QC,High_Mito,Low_nFeature,Low_total_count,Pass
mt_high_outlier,,,,
False,0,85230,189901,2493304
True,139239,983,103400,0


In [12]:
pd.crosstab(adata.obs['majority_voting_Level1'],adata.obs['mt_high_outlier'])

mt_high_outlier,False,True
majority_voting_Level1,,
Erythroid,267578,10630
Haematopoietic,198949,7840
Mesenchymal,1499714,156845
Neural,391784,21780
Stroma,410410,46527


In [13]:
pct = (
    pd.crosstab(
        adata.obs['Technology'].copy(),
        adata.obs['mt_high_outlier'].copy(),
        normalize='index'
    )
    * 100
)

# rename the columns for clarity
pct.columns = ['Non‐Outlier (%)', 'Mito‐Outlier (%)']


In [14]:
adata=adata[adata.obs['QC']=="Pass"].copy()

In [15]:
adata.obs['QC'].value_counts()

QC
Pass    2493304
Name: count, dtype: int64

In [16]:
adata.write_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_filtered_with_metadata.h5ad')

... storing 'QC' as categorical


In [1]:
import scanpy as sc 
import pandas as pd 
import helper_functions
adata= sc.read_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_filtered_with_metadata.h5ad')


In [2]:
adata= adata[(adata.obs['n_genes_by_counts']>200) & (adata.obs['total_counts']<60000)& (adata.obs['total_counts']>500) ].copy()

In [5]:
adata.write_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_filtered_with_metadata.h5ad')

In [4]:
adata.obs['10X_Chemistry'].value_counts()

10X_Chemistry
10x3'v2         804032
10x_multiome    668721
10x5'v1         106455
Name: count, dtype: int64

In [6]:
adata

AnnData object with n_obs × n_vars = 1579208 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_pf', 'S